In [1]:
import torch
from torch import nn

import fastai
from fastai.vision.all import (
    ImageDataLoaders,
    Resize,
    create_body,
    Learner,
    CrossEntropyLossFlat,
    accuracy_multi,
    accuracy,
    cnn_learner,
    # resnet18,
    # resnet34,
    resnet34,
    # resnet50,
    # resnet101,
    # resnet152,
    # densenet121,
    # densenet169,
    # densenet201,
    # densenet161,
    # vgg11,
    # vgg13,
    # vgg16,
    # vgg,
)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# fastai.vision.data.device = torch.device("cpu")

In [2]:
print(torch.cuda.is_available())

True


In [3]:
class_names = np.load("class_names.npy", allow_pickle=True)
# print(class_names.item())

attributes = np.load("attributes.npy", allow_pickle=True)
attributes.shape

(200, 312)

In [4]:
# Replace with your dataset path
df = pd.read_csv("train_images.csv")
dls = ImageDataLoaders.from_df(
    df,
    fn_col="image_path",
    label_col="label",
    item_tfms=Resize(224),
)

In [5]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy, accuracy_multi])

/home/smwhering/miniconda3/envs/aml/lib/python3.10/site-packages/fastai/vision/learner.py:301: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")


In [6]:
learn.fit_one_cycle(15)

epoch,train_loss,valid_loss,accuracy,accuracy_multi,time


AssertionError: Exception occured in `Recorder` when calling event `after_batch`:
	==:
12800
64

In [ ]:
learn.show_results()

## Using attributes.npy


In [ ]:
# Custom model architecture
class CustomModel(nn.Module):
    def __init__(self, num_classes=200, num_attributes=312):
        super(CustomModel, self).__init__()
        # Image processing branch (e.g., ResNet)
        self.image_branch = create_body(resnet34, pretrained=True)

        # Attribute processing branch
        self.attribute_branch = nn.Sequential(
            nn.Linear(num_attributes, 512), nn.ReLU(), nn.Linear(512, 512), nn.ReLU()
        )

        # Combining both branches
        self.classifier = nn.Linear(
            1024, num_classes
        )  # Assuming output of both branches is 512 each

    def forward(self, x, attr):
        # Image branch
        img_features = self.image_branch(x)
        img_features = nn.Flatten()(img_features)

        # Attribute branch
        attr_features = self.attribute_branch(attr)

        # Combine features
        combined = torch.cat((img_features, attr_features), dim=1)

        # Final classifier
        out = self.classifier(combined)
        return out


# Instantiate the model
model = CustomModel()

# Modify the DataLoaders to include attributes
# This requires custom implementation to ensure each batch contains image data and corresponding attributes

# Training loop will also need customization to handle both inputs

# Create learner
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

## Testing


In [ ]:
test_df = pd.read_csv("test_images_path.csv")  # Replace with the path to your test CSV
test_dls = dls.test_dl(
    test_df, folder="test_images", with_labels=True, bs=64
)  # Replace 'test_images' with your test images folder if different
preds, targets = learn.get_preds(dl=test_dls)

In [ ]:
predicted_labels = np.argmax(preds, axis=1)
acc = accuracy_score(targets, predicted_labels)
print(f"Accuracy: {acc * 100:.2f}%")

## Creating Sample


In [ ]:
# Create a submission DataFrame
submission_df = test_df.copy()
submission_df["label"] = predicted_labels

# Optional: if your test_df contains a column that represents an ID, use that instead
# For example, if there's an 'image_id' column:
# submission_df = pd.DataFrame({"id": test_df["image_id"], "label": predicted_labels})

# Save the submission file
submission_df[["id", "label"]].to_csv("submission.csv", index=False)

In [ ]:
torch.cuda.empty_cache()